### Remove DB if it Exists
In this tutorial we will create a database called example.db. We will use this step to delete it if it has already been created by a past run.

We will also set up a helper function that we can use later to show the contents of the database.

In [ ]:
import os

# Delete the db file if it exists.
if os.path.exists('example.db'):
    os.remove('example.db')

In [ ]:
def show_database(db_session):
    print('----------------------------------------------------------------------------')
    print('Instances:')
    for parameterized_instance in db_session.query(InstanceModel).all():
        print(f'\t{parameterized_instance}')
    
    print('Parameters:')
    for parameter in db_session.query(ParamModel).all():
        print(f'\t{parameter}\n\t\t{parameter.value}')
    print('----------------------------------------------------------------------------')
    

### Setup DB

To demonstrate the use of this library we are going to create a simple file sqlite database. We will create that here and add the connection, transaction and session for later use.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from param_persist.sqlalchemy.models import Base


engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

connection = engine.connect()
transaction = connection.begin()
session = Session(bind=connection)

### Param Class

Here we are setting up a basic param class. This class will be used throughout the example to show how to use the `param_persist` library.

In [ ]:
import param
import numpy as np

class ExampleParamClass(param.Parameterized):
    number_field = param.Number(0.5, doc="A simple number field.")
    integer_field = param.Integer(1, doc="A simple integer field.")
    string_field = param.String("My String", doc="A simple string field.")
    bool_field = param.Boolean(False, doc="A simple boolean field.")
    list_field = param.List([1,2,3,4], doc="A simple list field.")
    

### Create Agent

First we must create an agent. The agent is the worker for this library. Everything you will do you will do through the agent. Currently there is only an SqlAlchemyAgent. Custom agents can be written by overwrite the `AgentBase` class.

In [ ]:
from param_persist.agents.sqlalchemy_agent import SqlAlchemyAgent
from param_persist.sqlalchemy.models import InstanceModel, ParamModel

agent = SqlAlchemyAgent(engine=engine)

### Save a Parameterized Object

Next we will demonstrate how to save a paramaterized instance to the database. We will first show the contents of the database to show that the file table is empty. We will then create a parameterized instance that we will be working with and save it to the DB. Finally we will show the database contents again, showing that we have successfully persisted the data from the parameterized instance.

**Note:** The save method returns the ID of the instance row in the instances table. This will be used for interacting with that instance using the agent.

In [ ]:
# Demonstrate the table is empty
show_database(session)

# Create a parameterized instance and save it to the database
my_param = ExampleParamClass()
my_param_id = agent.save(my_param)

# Show the data has been persisted
show_database(session)


### Update a Paramterized Object

Here we will use the ID that we were given from the Save method to update the data in the database after changing the parameterized instance. We will first update some fields of the parameterized instance, then use the agents `update` method to update the database.

**NOTE:** We will update both `number_field` and `bool_field`. When the database is shown you will see the changes made to those fields.

In [ ]:
# Update the parameterized instance
my_param.number_field = 2.2
my_param.bool_field = True
my_param.list_field = [2,3,5,6,7]

# Use the agents update method
agent.update(my_param, my_param_id)

# Show the changed values in the database tables
show_database(session)

### Load Parameterized Object

In the next example we will show how to load a parameterized instance from the database using the agents `load` method using the `instance_id` saved in the database. Here we will create a new object using the `load` method and then show the values match what was saved in the above examples.

In [ ]:
# Load the param from the database
new_loaded_param = agent.load(my_param_id)

# Show the values
print(f'{new_loaded_param.bool_field} == True')
print(f'{new_loaded_param.number_field} == 2.2')
print(f'{new_loaded_param.integer_field} == 1')
print(f'{new_loaded_param.string_field} == My String')
print(f'{new_loaded_param.list_field} == [2,3,4,5,7]')

### Delete Parameterized Object

Finally we will show how to remove an instance from the database using the agents `delete` method. We will first show the database has values. Then we will use the `param_id` we have been using through the examples to delete the instance. Then we will show the database is empty.

**Note:** Deleting the instance will delete all of the parameters that it is linked too.

In [ ]:
# Show the database has data
show_database(session)

# Delete the parameterized instance from the database
agent.delete(my_param_id)

# Show the database is empty
show_database(session)